In [226]:
import os
import librosa
import numpy as np
import librosa.display
import soundfile as sf
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Flatten, Dense

In [5]:
import sounddevice as sd

In [6]:


def extract_features(audio_file):
    # Load audio
    audio, sample_rate = librosa.load(audio_file, sr=16000)
    
    # Extract MFCCs
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=13)
    
    # Mean normalize the MFCCs
    mfccs_normalized = mfccs.mean(axis=1)
    
    return mfccs_normalized

In [7]:
Prashasst_P1_01="E:/New-Codes/Repositories/IdentifyMe-AI/VoiceData/P1/Prashasst/Prashasst_P1_01.wav"

In [8]:
pras_norm=extract_features(Prashasst_P1_01)

In [9]:
type(pras_norm)

numpy.ndarray

In [10]:
print(pras_norm.shape)

(13,)


0:"Prashasst",1:"Satendra",2:"Shraddha",3:"Shreyansh",4:"Sourabh"

In [11]:
names={}
ylst=[]
index=0

In [12]:
folder_path = "E:/New-Codes/Repositories/IdentifyMe-AI/VoiceData/P1" 
final=[]
for name in os.listdir(folder_path):
    names[index]=name
    
    
    file_path=f"{folder_path}/{name}/"
    # print(file_path)
    for wav_file in os.listdir(file_path):
        # print(f"{file_path}{wav_file}")
        final.append(extract_features(f"{file_path}{wav_file}"))
        ylst.append(index)
    index+=1
# X=np.array(final)

In [13]:
names

{0: 'Prashasst',
 1: 'Sarthak',
 2: 'Satendra',
 3: 'Shraddha',
 4: 'Shreyansh',
 5: 'Sourabh'}

In [14]:
ylst

[0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 5]

In [15]:
X=np.array(final)
y=np.array(ylst)

In [16]:

# y= np.array([ i for i in range(5) for _ in range(5)])
print(f"X shape {X.shape}  y shape {y.shape}")

X shape (30, 13)  y shape (30,)


In [17]:
np.save("final.npy",X)

In [227]:

kf = KFold(n_splits=10, shuffle=True, random_state=42) 

In [18]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=49)

In [19]:
X_val, X_test, y_val, y_test = train_test_split(X_train, y_train, test_size=0.4, random_state=63)

In [20]:
X_train.shape

(20, 13)

In [21]:
# Define the model in a single line using Sequential
model = Sequential([
    InputLayer(input_shape=(13,)),  # Input layer
    # Flatten(),                          # Flatten the input
    Dense(120,activation="relu"),
    Dense(units=81, activation="relu",kernel_regularizer=tf.keras.regularizers.L2(l2=0.7)),
    Dense(units=64, activation='relu',kernel_regularizer=tf.keras.regularizers.L2(l2=0.5)),      # First hidden layer
    Dense(32, activation='relu',kernel_regularizer=tf.keras.regularizers.L2(l2=0.7)),      # Second hidden layer
    Dense(len(names), activation='softmax')      # Output layer for binary classification
])





e:\New-Codes\Repositories\IdentifyMe-AI\venv\Lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [234]:
id=0
for train_index, val_index in kf.split(X):

    X_train, X_val = X[train_index], X[val_index]

    y_train, y_val = y[train_index], y[val_index]


    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Use sparse categorical crossentropy
              metrics=['accuracy'])  # You can also use ['accuracy', 'precision', 'recall'] if needed
    history = model.fit(X_train, y_train,
                    epochs=25,            # Adjust the number of epochs as needed
                    batch_size=32,        # Adjust based on your data size
                    validation_data=(X_val, y_val))
    id+=1
    model.save(f"k_fold_{id}.keras")
    y_pred=model.predict(X_test)

    threshold = 0.6  # Example threshold
    predicted_classes = []

    for i in range(len(y_pred)):
        # Check if the highest probability exceeds the threshold
        if np.max(y_pred[i]) > threshold:
            predicted_classes.append(np.argmax(y_pred[i]))
        else:
            predicted_classes.append(-1)  # Or some indication of 'no confident prediction'
    
    print(f" model {id} : {predicted_classes}")
    print(y_test)
    # y_pr=model.predict()

Epoch 1/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 1.0000 - loss: 0.4297 - val_accuracy: 1.0000 - val_loss: 0.7729
Epoch 2/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - accuracy: 0.9259 - loss: 0.5646 - val_accuracy: 1.0000 - val_loss: 0.4411
Epoch 3/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 1.0000 - loss: 0.4402 - val_accuracy: 1.0000 - val_loss: 0.4314
Epoch 4/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 1.0000 - loss: 0.4592 - val_accuracy: 1.0000 - val_loss: 0.4665
Epoch 5/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 1.0000 - loss: 0.4944 - val_accuracy: 1.0000 - val_loss: 0.4474
Epoch 6/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 1.0000 - loss: 0.4592 - val_accuracy: 1.0000 - val_loss: 0.4497
Epoch 7/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 1.0000 - loss: 0.4363 - val_accuracy: 1.0000 - val_loss: 0.4815
Epoch 8/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 1.0000 - loss: 0.4512 - val_accuracy: 1.0000 - val_loss: 0.4848
Ep

In [22]:
# Compile the model
# Compile the model for multi-class classification
# model.compile(optimizer='adam',
#               loss='sparse_categorical_crossentropy',  # Use sparse categorical crossentropy
#               metrics=['accuracy'])  # You can also use ['accuracy', 'precision', 'recall'] if needed


# Summary of the model
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 120)            │         1,680 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 81)             │         9,801 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         5,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 6)              │           198 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,007 (74.25 KB)

 Trainable params: 19,007 (74.25 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
history = model.fit(X_train, y_train,
                    epochs=500,            # Adjust the number of epochs as needed
                    batch_size=32,        # Adjust based on your data size
                    validation_data=(X_val, y_val))

Epoch 1/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.1500 - loss: 153.8521 - val_accuracy: 0.1667 - val_loss: 147.9887
Epoch 2/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.1500 - loss: 146.7240 - val_accuracy: 0.1667 - val_loss: 141.6220
Epoch 3/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.1500 - loss: 140.2848 - val_accuracy: 0.3333 - val_loss: 136.8226
Epoch 4/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.2000 - loss: 135.4034 - val_accuracy: 0.2500 - val_loss: 133.0768
Epoch 5/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.3500 - loss: 131.7681 - val_accuracy: 0.0000e+00 - val_loss: 130.1993
Epoch 6/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.1000 - loss: 129.2092 - val_accuracy: 0.0833 - val_loss: 128.3098
Epoch 7/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.1500 - loss: 127.3595 - val_accuracy: 0.0833 - val_loss: 126.4399
Epoch 8/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.2000 - loss: 125.5864 - 

In [24]:
model.save("2nd_reg_model_new_ach_epocs.keras")

In [25]:
X_test.shape

(8, 13)

In [26]:
# p=model.predict(X_test[0].reshape(-1,13))

In [233]:
for id in range(1,11):
    mod=f"k_fold_{id}.keras"

    new_model = tf.keras.models.load_model(mod)
    y_pred=model.predict(X_test)

    threshold = 0.6  # Example threshold
    predicted_classes = []

    for i in range(len(y_pred)):
        # Check if the highest probability exceeds the threshold
        if np.max(y_pred[i]) > threshold:
            predicted_classes.append(np.argmax(y_pred[i]))
        else:
            predicted_classes.append(-1)  # Or some indication of 'no confident prediction'
    
    print(f" model {id} : {predicted_classes}")
print(y_test)


# Show the model architecture
    # new_model.summary()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
 model 1 : [4, 0, 1, 3, 3, 0, 5, 4]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
 model 2 : [4, 0, 1, 3, 3, 0, 5, 4]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
 model 3 : [4, 0, 1, 3, 3, 0, 5, 4]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
 model 4 : [4, 0, 1, 3, 3, 0, 5, 4]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
 model 5 : [4, 0, 1, 3, 3, 0, 5, 4]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
 model 6 : [4, 0, 1, 3, 3, 0, 5, 4]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
 model 7 : [4, 0, 1, 3, 3, 0, 5, 4]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
 model 8 : [4, 0, 1, 3, 3, 0, 5, 4]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
 model 9 : [4, 0, 1, 3, 3, 0, 5, 4]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
 model 10 : [4, 0, 1, 3, 3, 0, 5, 4]
[4 0 1 3 3 0 5 4]


In [27]:
pred=model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step


In [28]:
y_pred=pred

In [29]:
# names

In [30]:
threshold = 0.6  # Example threshold
predicted_classes = []

for i in range(len(y_pred)):
    # Check if the highest probability exceeds the threshold
    if np.max(y_pred[i]) > threshold:
        predicted_classes.append(np.argmax(y_pred[i]))
    else:
        predicted_classes.append(-1)  # Or some indication of 'no confident prediction'


In [31]:
predicted_classes

[4, 0, 1, 3, 3, 0, -1, 4]

In [32]:
y_test

array([4, 0, 1, 3, 3, 0, 5, 4])

In [174]:


# Parameters for recording
duration = 4  # seconds
sample_rate = 16000  # Sample rate

# Function to capture audio
def record_audio(duration, sample_rate):
    print("Recording...")
    audio_data = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='float32')
    sd.wait()  # Wait until the recording is finished
    print("Recording finished.")
    return np.squeeze(audio_data)  # Convert to 1D array

# Record 5 seconds of audio
audio1 = record_audio(duration, sample_rate)

print("Captured Audio Shape:", audio1.shape)


Recording...
Recording finished.
Captured Audio Shape: (64000,)


In [167]:
audio1.shape

(64000,)

In [179]:
def extract_mfcc(audio, sample_rate=16000, n_mfcc=13):
    # Extract MFCCs from the raw audio signal
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=n_mfcc)
    # librosa returns MFCCs as (n_mfcc, n_frames). We can take the mean over time to get a fixed size (n_mfcc,)
    # mfccs_mean = np.mean(mfccs, axis=1)
    # return mfccs_mean
    mfccs_normalized = mfccs.mean(axis=1)
    
    return mfccs_normalized, mfccs

In [184]:
def extract_mfcc(audio_file):
    # Load audio
    audio, sample_rate = librosa.load(audio_file, sr=16000)
    
    # Extract MFCCs
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=13)
    
    # Mean normalize the MFCCs
    mfccs_normalized = mfccs.mean(axis=1)
    
    return mfccs_normalized , mfccs

In [213]:
folder_path = "E:/New-Codes/Repositories/IdentifyMe-AI/VoiceData/Test/" 
test1=[]
wavs=[]
for wav_file in os.listdir(folder_path):
    # print(f"{folder_path}{wav_file}")
    norm, mfccs=extract_mfcc(f"{folder_path}/{wav_file}")
    test1.append(norm)
    print(mfccs.shape)
    wavs.append(mfccs)

(13, 97)
(13, 207)
(13, 87)
(13, 157)
(13, 105)


In [214]:
test=np.array(test1)
# wavs1=np.array(wavs)

In [215]:
# wavs.

In [180]:
pras_norm2,yoyo= extract_mfcc(audio1)

In [181]:
print(f" norm shape {pras_norm2.shape} mfccs shape {yoyo.shape}")

 norm shape (13,) mfccs shape (13, 126)


In [216]:
# import numpy as np
# import librosa
# import librosa.display
# import soundfile as sf

# Function to invert the MFCC to the Mel-spectrogram
def mfcc_to_mel(mfcc, sr, n_mfcc=13):
    # librosa's default for n_mels is 128. If your MFCC calculation used different settings, adjust accordingly
    mel_basis = librosa.filters.mel(sr=sr, n_fft=2048, n_mels=128)
    inv_mfcc = librosa.feature.inverse.mfcc_to_mel(mfcc, n_mels=128, dct_type=2)
    return inv_mfcc

# Function to invert the Mel-spectrogram to the audio waveform using Griffin-Lim
def mel_to_audio(mel_spectrogram, sr):
    # Convert the Mel spectrogram back to waveform using Griffin-Lim algorithm
    waveform = librosa.feature.inverse.mel_to_audio(mel_spectrogram, sr=sr, n_fft=2048, hop_length=512)
    return waveform




In [225]:
# Example MFCC array
sr = 16000  # Sample rate
mfcc = wavs[0]  # Example MFCC (should be your actual MFCC array)

# Invert MFCC to Mel spectrogram
mel_spectrogram = mfcc_to_mel(mfcc, sr)

# Invert Mel spectrogram to audio
reconstructed_audio = mel_to_audio(mel_spectrogram, sr)

# Save the reconstructed audio
sf.write('reconstructed.wav', reconstructed_audio, sr)

In [218]:
test.shape

(5, 13)

In [219]:
# ans=model.predict(pras_norm2.reshape(-1,13))
ans=model.predict(test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


In [220]:
# ans.reshape(len(names),-1)
ans.shape

(5, 6)

In [221]:
ans

array([[0.6145154 , 0.18905573, 0.01449946, 0.02025566, 0.12781766,
        0.03385612],
       [0.00842796, 0.37047604, 0.01146543, 0.28279668, 0.01381356,
        0.3130203 ],
       [0.01851486, 0.75281286, 0.00366993, 0.11082861, 0.01125874,
        0.10291505],
       [0.13315557, 0.1133599 , 0.15668397, 0.02582505, 0.3702204 ,
        0.20075509],
       [0.00964895, 0.00316884, 0.6000048 , 0.00294939, 0.28756574,
        0.09666229]], dtype=float32)

In [222]:
threshold = 0.6  # Example threshold
ans1 = []

    # Check if the highest probability exceeds the threshold
# if np.max(ans) > threshold:
#     ans1.append(np.argmax(ans))
# else:
#     ans1.append(-1)  # Or some indication of 'no confident prediction'



for i in range(len(ans)):
    # Check if the highest probability exceeds the threshold
    if np.max(ans[i]) > threshold:
        ans1.append(np.argmax(ans[i]))
    else:
        ans1.append(-1)  # Or some indication of 'no confident prediction'


In [223]:
ans1

[0, -1, 1, -1, 2]

In [224]:
for i in ans1:
    if i==-1:print("Could not identify")
    else:print(f"Person identified as {names[i]}")

Person identified as Prashasst
Could not identify
Person identified as Sarthak
Could not identify
Person identified as Satendra


In [47]:
test.shape

(3, 13)